In [ ]:
import pandas as pd
from pprint import pprint
import sklearn
from sklearn.model_selection import train_test_split
# from sklearn.metrics import root_mean_squared_error, mean_absolute_error, r2_score

### Preprocessing

In [13]:


dataset = pd.read_csv("housing.csv")
# dataset.head()
X = dataset.drop(columns=["median_house_value", 'ocean_proximity'])
y = dataset['median_house_value']



pd.get_dummies(dataset['ocean_proximity'])
X = pd.concat([X, pd.get_dummies(dataset['ocean_proximity'])], axis=1)

X.head()
M, N = X.shape



X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


### Variable and function declarations

In [16]:
def predict(X, weights, bias):

    y = []

    for i in range(len(X.shape[0])):
        pred = 0
        for j in range(len(weights)):
            pred += weights[j]*X[i][j]
        pred += bias
        y.append(pred)
    return y


    

def mae_cost(X, y, weights, bias):
    m, n = X.shape
    if (n != len(weights)):
        raise Exception("numbers of weights and features are not same")
    
    cost = 0

    for i in range(m):
        Y_pred = 0
        for j in range(n):
            Y_pred += weights[j]*X[i][j]
        Y_pred += bias

        cost += abs(Y_pred, y[i])

    return cost/m
    
def mse_cost(X, y, weights, bias):
    m, n = X.shape

    cost = 0
    for i in range(m):
        y_pred = 0
        for j in range(n):
            try:
              y_pred += weights[j]*X[i][j] 
            except:
              print(i, ' ', j)
              raise
        y_pred += bias
        
        cost += (y_pred-y[i])**2

    return cost/2/m

def r_squared(X, y, weights, bias):
    m, n = X.shape

    S_res = 2*m*mse_cost(X, y, weights, bias)
    average=sum(y)/m

    S_tot = sum([(i-average)**2 for i in y])

    return (1-S_res/S_tot)


W = [0 for i in range(N)] #weights
b = 0  #bias




In [17]:
def one_step(X, y, weights, bias, alpha=1e-3):

    y_predicted = predict(X, weights, bias)
    m, n = X.shape

    weights = weights[:]
    

    for j in range(len(weights)):
        weights[j] = weights[j] - alpha*1/m*(sum([(y_predicted[i]-y[i])*X[i][j] for i in range(m)]))
    
    bias -= alpha * 1/m * sum([y_predicted[i]-y[i] for i in range(m)])

    return weights, bias


def gradient_descent(X, y, weights, bias, alpha=1e-3):
    m, n = X.shape


    mse_costs= [mse_cost(X, y, weights, bias)]
    # r2_score = [r_squared(X, y, weights, bias)]
    # mae_costs = [mae_cost(X, y, weights, bias)]




    while True:
        y_predicted = predict(X, weights, bias)

        n_w, n_b = one_step(X, y, weights, bias, alpha=alpha)

        mse_costs.append(mse_cost(X, y, n_w, n_b))
        # r2_score.append(r_squared(X, y, n_w, n_b))
        # mae_costs.append(mae_cost(X, y, n_w, n_b))

        if (all([(abs(weights[j]-n_w[j] )< 1e-9 for j in range(len(weights)))]) and abs(n_b-bias)<1e-9):
            weights, bias = n_w, n_b
            break;
        weights, bias = n_w, n_b
    

    
    return (mse_costs, mae_costs, r2_score), weights, bias



(mse_costs, mae_costs, r2_score), W, b = gradient_descent(X_test, y_test, weights=W, bias=b)




        



0   0


KeyError: 0